In [9]:
# !pip install all of this :
#chainlit==0.7.700
# cohere==4.37
# openai==1.23.6
# tiktoken==0.6.0
# python-dotenv==1.0.0
# langchain==0.1.16
# langchain-core==0.1.46
# langchain-community==0.0.34
# langchain-openai==0.1.4
# qdrant-client==1.9.0
# pymupdf==1.24.2

In [6]:
import chainlit as cl
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import tiktoken
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter
from langchain.retrievers.multi_query import MultiQueryRetriever
import os
import openai
from getpass import getpass
!pip install pymupdf

In [4]:
openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [8]:
openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")
openai_chat_model_4 = ChatOpenAI(model="gpt-4-turbo")


def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-3.5-turbo").encode(
        text,
    )
    return len(tokens)

docs = PyMuPDFLoader("Meta10k.pdf").load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50,
    length_function = tiktoken_len,
)

split_chunks = text_splitter.split_documents(docs)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

qdrant_vectorstore = Qdrant.from_documents(
    split_chunks, 
    embedding_model, 
    location=":memory:",
    collection_name="Meta10k",
)

# THE SECRET SAUCE
qdrant_retriever = qdrant_vectorstore.as_retriever(search_type="mmr", search_kwargs={'k': 6, 'lambda_mult': 0.25})

2024-05-01 20:23:43 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-01 20:23:45 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-01 20:23:47 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-01 20:23:49 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [10]:
RAG_PROMPT = """
You are an expert financial analyst.  You will be provided CONTEXT excerpts from the META company 10K annual report.  Your job is to answer the QUERY as correctly as you can using the information provided by the CONTEXT and your skills as an expert financial analyst.  For questions regarding money do not over think it and begin adding values unless you are specifically asked to.  Use the simplest most obvious choice. IF the context provided does give you enough information to answer the question, respond "I do not know"

CONTEXT:
{context}

QUERY:
{question}
"""

EVAL_SYSTEM_TEMPLATE = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response and the provided context
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following format:

"""

EVAL_USER_TEMPLATE = """Query: {input}
Context: {context}
Response: {response}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)
eval_prompt = ChatPromptTemplate.from_messages([
    ("system", EVAL_SYSTEM_TEMPLATE),
    ("human", EVAL_USER_TEMPLATE)
])

retriever_from_llm = MultiQueryRetriever.from_llm(retriever=qdrant_retriever, llm=openai_chat_model)

chain = ({"context": itemgetter("question") | retriever_from_llm, "question": itemgetter("question")} | RunnablePassthrough.assign(context=itemgetter("context")) | {"response": rag_prompt | openai_chat_model, "context": itemgetter("context")})
eval_chain = eval_prompt | openai_chat_model_4

In [13]:
#Question #1

response = chain.invoke({"question":"What was the total value of 'Cash and cash equivalents' as of December 31, 2023?"})

context = "\n".join([context.page_content for context in response["context"]])
eval_response = eval_chain.invoke({"input":"What was the total value of 'Cash and cash equivalents' as of December 31, 2023?", "context":context, "response":response["response"].content})

2024-05-01 20:27:50 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-01 20:27:50 - Generated queries: ["1. Can you provide the total amount of 'Cash and cash equivalents' on December 31, 2023?", "2. How much was the value of 'Cash and cash equivalents' on December 31, 2023?", "3. What was the total worth of 'Cash and cash equivalents' as at December 31, 2023?"]
2024-05-01 20:27:50 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-01 20:27:50 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-01 20:27:50 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-01 20:27:52 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-01 20:28:00 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [16]:
# Response from LLM
response["response"].content

"The total value of 'Cash and cash equivalents' as of December 31, 2023, was $41,862 million."

In [15]:
# LLM evaluating my original response
eval_response.content

"Clarity: 9/10\nThe response is clear and concise, directly stating the total value of 'Cash and cash equivalents' as of December 31, 2023. It provides a specific figure which is easily understandable.\n\nFaithfulness: 10/10\nThe response is completely faithful to the original query and the provided context. It precisely addresses the query by extracting the exact figure from the context without adding any unrelated information.\n\nCorrectness: 10/10\nThe response is correct as it accurately reports the value of 'Cash and cash equivalents' as $41,862 million, which matches the data provided in the context. \n\nOverall, the response effectively communicates the required information and is both accurate and relevant to the query."

In [18]:
#Question #2
response = chain.invoke({"question":"Who are Meta's 'Directors' (i.e., members of the Board of Directors)?"})

context = "\n".join([context.page_content for context in response["context"]])
eval_response = eval_chain.invoke({"input":"Who are Meta's 'Directors' (i.e., members of the Board of Directors)?", "context":context, "response":response["response"].content})

2024-05-01 20:31:32 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-01 20:31:32 - Generated queries: ["1. Can you provide a list of individuals who serve as 'Directors' at Meta, also known as members of the Board of Directors?", "2. Who makes up the group of 'Directors' at Meta, specifically referring to the members of the Board of Directors?", "3. Could you share the names of the individuals who hold the title of 'Directors' at Meta, in other words, the members of the Board of Directors?"]
2024-05-01 20:31:32 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-01 20:31:32 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-01 20:31:32 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-01 20:31:35 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-01 20:31:46 - HTTP Request: POST https://api.openai.com/v1/chat/

In [20]:
# Response from LLM for question #2
response["response"].content


"The members of Meta's Board of Directors, as listed in the document, include:\n1. Mark Zuckerberg\n2. Susan Li\n3. Aaron Anderson\n4. Peggy Alford\n5. Marc L. Andreessen\n6. Andrew W. Houston\n7. Nancy Killefer\n8. Robert M. Kimmitt\n9. Sheryl K. Sandberg\n10. Tracey T. Travis\n11. Tony Xu"

In [21]:
#Eval of response #2
eval_response.content

"1. Clarity - 9/10\n   - The response is quite clear and lists the members of Meta's Board of Directors succinctly. Each name is presented in an easy-to-read format.\n\n2. Faithfulness - 10/10\n   - The response accurately captures and relays information directly relevant to the query about the members of Meta's Board of Directors. It correctly uses the information provided in the context.\n\n3. Correctness - 6/10\n   - The response inaccurately includes Susan Li and Aaron Anderson as members of Meta's Board of Directors. Susan Li is listed as the Chief Financial Officer and Aaron Anderson as the Chief Accounting Officer in the document, not as board members. The correct response should have excluded these two names from the list of directors."